In [1]:
# imports
from sqlalchemy import create_engine, Column, String, Integer, Numeric, MetaData, Table, type_coerce
from sqlalchemy.orm import mapper, create_session
import csv
from sqlalchemy import case
import numpy as np
from sqlalchemy.ext.automap import automap_base
import matplotlib.pyplot as plt
import sqlalchemy.types as types
import numbers
from sqlalchemy.sql import and_, or_, not_
from sqlalchemy.sql import select
%matplotlib inline

In [2]:
# definitions
file1='/Users/eolson/WorkUBC/MEOPAR/obs/BioChem/BioChem_Query_1610/BioChem_Query_1610_Data.csv'

In [3]:
engine = create_engine('sqlite:////Users/eolson/WorkUBC/MEOPAR/obs/BioChem/BioChem_Query_1610/BioChemQueryDB.sqlite')

In [4]:
table = None
metadata = MetaData(bind=engine)


In [5]:
class forceNumeric(types.TypeDecorator):

    impl = types.Numeric

    def process_bind_param(self, value, dialect):
        try:
            int(float(value))
        except:
            value = None
        return value

class forceInt(types.TypeDecorator):
    
    impl = types.Integer
    def process_bind_param(self, value, dialect):
        try:
            int(value)
        except:
            value = None
        return value


In [6]:
def coltype(ikey):
    typedict = {
        'DATA_CENTER': String(),
        'DESCRIPTOR-NAME': String(),
        'AREA_NAME': String(),
        'COLLECTOR_STATION_NAME': String(),
        'START_DATE': String(),
        'START_TIME': String(),
        'COLLECTOR_DEPLOYMENT_ID': String(),
        'GEAR_MODEL (GEAR_SIZE)': String(),
        'Silicate_datatype': String(),
        'Phosphate_datatype': String(),
        'Nitrate_datatype': String(),
    }
    return typedict.get(ikey, forceNumeric())
    

In [7]:
with open(file1) as f:
    cf = csv.DictReader(f, delimiter=',')
    for row in cf:
        if table is None:
            table = Table('profiles', metadata, Column('id', Integer, primary_key=True),
                *(Column(rowname, coltype(rowname)) for rowname in row.keys()))
            table.create()
        table.insert().values(**row).execute()
class Profs(object): pass
mapper(Profs, table)

<Mapper at 0x10d702390; Profs>

In [8]:
Base = automap_base()
# reflect the tables in salish.sqlite:
Base.prepare(engine, reflect=True)
# mapped classes have been created
Profs=Base.classes.profiles
session = create_session(bind = engine, autocommit = False, autoflush = True)


In [13]:
print(row.keys())

dict_keys(['COLLECTOR_STATION_NAME', 'START_TIME', 'Silicate', 'SOUNDING', 'COLLECTOR (RESPONSIBLE_GROUP)', 'START_DATE', 'DESCRIPTOR-NAME', 'START_DEPTH', 'COLLECTOR_DEPLOYMENT_ID', 'Silicate_qcCode', 'END_DEPTH', 'Phosphate_qcCode', 'COLLECTOR_SAMPLE_ID', 'Nitrate_datatype', 'START_LON', 'START_LAT', 'QRY_NO', 'Nitrate', 'AREA_NAME', 'DATA_CENTER', 'Nitrate_qcCode', 'Silicate_datatype', 'Phosphate', 'Phosphate_datatype', 'COLLECTOR_EVENT_ID', 'GEAR_MODEL (GEAR_SIZE)'])


In [17]:
data=session.query(#Profs.START_DATE,Profs.START_DEPTH,Profs.START_LAT,Profs.START_LON, 
                  Profs.Nitrate, Profs.Silicate, Profs.Phosphate
                  ).filter(or_(
                           Profs.Nitrate != None,
                           Profs.Phosphate != None,
                           Profs.Silicate != None,
                          )).all()
print(data)


[(None, Decimal('96.2000000000'), Decimal('3.7200000000')), (Decimal('0.3000000000'), Decimal('3.1000000000'), Decimal('0.3700000000')), (Decimal('1.1000000000'), Decimal('4.0000000000'), Decimal('0.3300000000')), (None, Decimal('97.9000000000'), Decimal('4.7500000000')), (Decimal('19.1000000000'), Decimal('77.5000000000'), Decimal('3.2800000000')), (Decimal('26.5000000000'), Decimal('62.8000000000'), Decimal('2.1800000000')), (Decimal('28.7000000000'), Decimal('59.0000000000'), Decimal('2.5200000000')), (Decimal('26.3000000000'), Decimal('47.7000000000'), Decimal('2.3800000000')), (Decimal('17.8000000000'), Decimal('29.7000000000'), Decimal('1.8400000000')), (Decimal('26.1000000000'), Decimal('65.4000000000'), Decimal('2.3700000000')), (Decimal('22.4000000000'), Decimal('85.2000000000'), Decimal('2.5400000000')), (Decimal('18.1000000000'), Decimal('32.4000000000'), Decimal('1.6800000000')), (Decimal('15.4000000000'), Decimal('27.5000000000'), Decimal('1.6100000000')), (Decimal('13.600

In [19]:
session.close()
engine.dispose()